In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import *

1.8.1
0.8.0a0+e4e171a


In [16]:
cfg = Config(
    noise_nsr_dbs = [20, 10, 3],
    site='SSW', 
    use_neptune=True, 
    n_epochs=10,
    bs=32, 
    lr=1e-3, 
    model='densenet-mixedspec',
)

In [17]:
cfg.as_dict()

{'site': 'SSW',
 'data_dir': 'data/birdclef-2021',
 'checkpoint_dir': 'checkpoints',
 'bs': 32,
 'n_workers': 12,
 'training_dataset_size': 10752,
 'duration': 5,
 'use_noise': True,
 'noise_nsr_dbs': [20, 10, 3],
 'noise_dir': 'data/noise/BirdVox-DCASE-20k/wav-32k',
 'use_overlays': True,
 'max_overlays': 5,
 'overlay_weights': [0.71986223,
  0.21010333,
  0.06314581,
  0.00574053,
  0.00114811],
 'overlay_snr_dbs': [20, 10, 3],
 'use_neptune': True,
 'sr': 32000,
 'n_frames': 160000,
 'n_fft': 512,
 'window_length': 512,
 'n_mels': 64,
 'hop_length': 312,
 'f_min': 150,
 'f_max': 15000,
 'lr': 0.001,
 'n_epochs': 10,
 'model': 'densenet-mixedspec',
 'optimizer': 'torch.optim.Adam',
 'loss_fn': 'torch.nn.BCEWithLogitsLoss',
 'scheduler': 'torch.optim.lr_scheduler.CosineAnnealingLR',
 'labels': ['amecro',
  'amegfi',
  'amerob',
  'balori',
  'belkin1',
  'bkcchi',
  'blujay',
  'cangoo',
  'chswar',
  'comgra',
  'comyel',
  'dowwoo',
  'eastow',
  'eawpew',
  'gockin',
  'grycat',
  

In [21]:
prep = torch.nn.Sequential(
    MixedSig2Spec(cfg),
)

main_model = torch.hub.load('pytorch/vision:v0.9.0', 'densenet121', pretrained=True)
main_model.classifier = torch.nn.Linear(in_features=1024, out_features=len(cfg.labels), bias=True)

model = torch.nn.Sequential(prep, main_model)
model = model.cuda()

Using cache found in /home/koen/.cache/torch/hub/pytorch_vision_v0.9.0


In [22]:
lrn = Learner("densenet-densenet-mixed-spec", cfg, model)

In [23]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-40


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:   9 train loss: 0.111053 train f1: 0.574801 valid loss: 0.125677 valid f1: 0.140071


  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:  10 train loss: 0.110937 train f1: 0.580079 valid loss: 0.106580 valid f1: 0.284351


In [24]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,381,29231,3599,389,0.160421
2,0.2,265,31949,881,505,0.276618
3,0.3,210,32437,393,560,0.305900
4,0.4,174,32607,223,596,0.298201
5,0.5,149,32701,129,621,0.284351
6,0.6,122,32756,74,648,0.252588
7,0.7,103,32793,37,667,0.226374
8,0.8,85,32810,20,685,0.194286
9,0.9,61,32828,2,709,0.146459


In [25]:
lrn.load_checkpoint()

{'epoch': 7,
 'valid_loss': 0.08945167613656897,
 'valid_score': 0.3130990266799927}

In [26]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,335,31385,1445,435,0.262745
2,0.2,233,32570,260,537,0.368963
3,0.3,189,32742,88,581,0.361032
4,0.4,170,32791,39,600,0.347293
5,0.5,147,32808,22,623,0.313099
6,0.6,119,32813,17,651,0.262693
7,0.7,99,32821,9,671,0.225513
8,0.8,80,32824,6,690,0.186916
9,0.9,49,32828,2,721,0.119367
